# Content-Based Recommender
### Credits, Genres, and Keywords Based Recommender
Suggest similar items based on a particular item. The general idea behind these recommender systems is that if a person likes a particular item, he or she will also like an item that is similar to it. The subset dataset can be downloaded from [here](https://www.kaggle.com/rounakbanik/the-movies-dataset/data).

In [1]:
import pandas as pd

In [2]:
# Load metadata, keywords and credits
credits = pd.read_csv('dataset/credits.csv')
keywords = pd.read_csv('dataset/keywords.csv')
metadata = pd.read_csv('dataset/movies_metadata.csv', low_memory=False)

In [3]:
credits.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [4]:
keywords.head(3)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


In [5]:
# Remove rows with bad IDs.
bad_row_mask = metadata['imdb_id'] == '0'
bad_rows =  metadata.loc[bad_row_mask]
bad_row_idx = bad_rows.index
metadata = metadata.drop(bad_row_idx)

In [6]:
# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [7]:
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [8]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [9]:
import numpy as np

In [10]:
def get_director(x):
    for item in x:
        if item['job'] == 'Director':
            return item['name']
    return np.nan

In [11]:
metadata['director'] = metadata['crew'].apply(get_director)
metadata['director']

0           John Lasseter
1            Joe Johnston
2           Howard Deutch
3         Forest Whitaker
4           Charles Shyer
               ...       
46623    Hamid Nematollah
46624            Lav Diaz
46625      Mark L. Lester
46626    Yakov Protazanov
46627       Daisy Asquith
Name: director, Length: 46628, dtype: object

In [12]:
def get_list(x):
    if isinstance(x, list):
        names = [ item['name'] for item in x]
        
        # Check if more than 3 elements exist. 
        # If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names
    
    return []

In [13]:
features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [14]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


The next step would be to convert the names and keyword instances into `lowercase` and `strip` all the spaces between them.
Removing the spaces between words is an important preprocessing step. It is done so that vectorizer doesn't count the Johnny of "Johnny Depp" and "Johnny Galecki" as the same. 

In [15]:
def remove_space(x):
    if isinstance(x, list):
        return [(i.replace(" ", "")) for i in x]
    elif isinstance(x, str):
        return x.replace(" ", "")
    else:
        return ''

In [16]:
def lower_string(x):
    if isinstance(x, list):
        return [str.lower(i) for i in x]
    elif isinstance(x, str):
        return str.lower(x)
    else:
        return ''

In [17]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(remove_space).apply(lower_string)

In [18]:
metadata['genres']

0         [animation, comedy, family]
1        [adventure, fantasy, family]
2                   [romance, comedy]
3            [comedy, drama, romance]
4                            [comedy]
                     ...             
46623                 [drama, family]
46624                         [drama]
46625       [action, drama, thriller]
46626                              []
46627                              []
Name: genres, Length: 46628, dtype: object

The `create_soup` function will simply join all the required columns by a space. This is the final preprocessing step, and the output of this function will be fed into the word vector model.

In [19]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [20]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)
metadata[['soup']].head(2)

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...


One key difference is that you use the `CountVectorizer()` instead of `TF-IDF`. This is because you do not want to down-weight the actor/director's presence if he or she has acted or directed in relatively more movies.

In [21]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

In [22]:
count_matrix.shape

(46628, 73881)

From the above output, you can see that there are 73,881 vocabularies in the metadata that you fed to it. Next, use the `cosine_similarity` to measure the distance between the embeddings.

In [23]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [24]:
movie_indices = pd.Series(metadata.index, index=metadata['title'])

In [25]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = movie_indices[title]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Top 10 
    sim_scores = sim_scores[1:11]
    
    # Get the movie indices
    indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar movies
    return metadata['title'].iloc[indices]

In [26]:
get_recommendations('The Dark Knight Rises', cosine_sim)

12589      The Dark Knight
10210        Batman Begins
9311                Shiner
9874       Amongst Friends
7772              Mitchell
516      Romeo Is Bleeding
11463         The Prestige
24090            Quicksand
25038             Deadfall
41063                 Sara
Name: title, dtype: object

In [27]:
get_recommendations('The Godfather', cosine_sim)

1934            The Godfather: Part III
1199             The Godfather: Part II
15609                   The Rain People
18940                         Last Exit
34488                              Rege
35802            Manuscripts Don't Burn
35803            Manuscripts Don't Burn
8001     The Night of the Following Day
18261                 The Son of No One
28683            In the Name of the Law
Name: title, dtype: object